# Scatter and scatter v

1. length: number of values to each processes
2. disp: starting location of data to each processes

Scatterv example:
 - x: 0 1 2 3 4 5 6
 - P0:
    - disp: 0
    - length: 2
    - start from x[0], send x[0] and x[1]
 - P1:
    - disp: 2
    - length: 2
    - start from x[2], send x[2] and x[3]
 - P2:
    - disp: 4
    - length: 1
    - start from x[4], send x[4]
 - P3:
    - disp: 5
    - length: 1
    - start from x[5], send x[5]

In [ ]:
%%writefile main_1.cpp
#include <mpi.h>
#include <iostream>
#include <unistd.h>

int main(int argc, char** argv) {
   MPI_Init(&argc, &argv);

   int myid, numprocs;
   MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
   MPI_Comm_rank(MPI_COMM_WORLD, &myid);

   if(myid == 0)
   {
      std::cout << "Number of processors: " << numprocs << std::endl;
   }

   if(numprocs != 4)
   {
      if(myid == 0)
      {
         std::cout << "Error: This program requires exactly 4 processors." << std::endl;
      }
      MPI_Finalize();
      return 1;
   }

   int sendcount[4] = {2,2,1,1};
   int displs[4] = {0,2,4,5};
   float x[10];
   for(int i = 0; i < 10; i++)
   {
      x[i] = i;
   }
   if(myid == 0)
   {
      std::cout << "Before Scatterv, processor " << myid << " has x = ";
      for(int i = 0; i < 10; i++)
      {
         std::cout << x[i] << " ";
      }
      std::cout << std::endl;
   }
   MPI_Barrier(MPI_COMM_WORLD);

   float *y = new float[sendcount[myid]];
   MPI_Scatterv(x, sendcount, displs, MPI_FLOAT, y, sendcount[myid], MPI_FLOAT, 0, MPI_COMM_WORLD);

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "After Scatterv, processor " << myid << " has y = ";
         for(int i = 0; i < sendcount[myid]; i++)
         {
            std::cout << y[i] << " ";
         }
         std::cout << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   MPI_Finalize();
   return 0;
}

In [ ]:
!mpicxx main_1.cpp -o main_1.ex

In [ ]:
!mpirun --oversubscribe --allow-run-as-root -np 4 ./main_1.ex

## Question:

What if we have to send

0 -> P0

3 4 -> P1

6 7 -> P2

9 -> p3

In [ ]:
%%writefile main_1.cpp
#include <mpi.h>
#include <iostream>
#include <unistd.h>

int main(int argc, char** argv) {
   MPI_Init(&argc, &argv);

   int myid, numprocs;
   MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
   MPI_Comm_rank(MPI_COMM_WORLD, &myid);

   if(myid == 0)
   {
      std::cout << "Number of processors: " << numprocs << std::endl;
   }

   if(numprocs != 4)
   {
      if(myid == 0)
      {
         std::cout << "Error: This program requires exactly 4 processors." << std::endl;
      }
      MPI_Finalize();
      return 1;
   }

   // TODO: How to modify here?
   // int sendcount[4] = {2,2,1,1};
   // int displs[4] = {0,2,4,5};
   // END TODO

   float x[10];
   for(int i = 0; i < 10; i++)
   {
      x[i] = i;
   }
   if(myid == 0)
   {
      std::cout << "Before Scatterv, processor " << myid << " has x = ";
      for(int i = 0; i < 10; i++)
      {
         std::cout << x[i] << " ";
      }
      std::cout << std::endl;
   }
   MPI_Barrier(MPI_COMM_WORLD);

   float *y = new float[sendcount[myid]];
   MPI_Scatterv(x, sendcount, displs, MPI_FLOAT, y, sendcount[myid], MPI_FLOAT, 0, MPI_COMM_WORLD);

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "After Scatterv, processor " << myid << " has y = ";
         for(int i = 0; i < sendcount[myid]; i++)
         {
            std::cout << y[i] << " ";
         }
         std::cout << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   MPI_Finalize();
   return 0;
}

In [ ]:
!mpicxx main_1.cpp -o main_1.ex

In [ ]:
!mpirun --oversubscribe --allow-run-as-root -np 4 ./main_1.ex

# Non-blocking Send and Recv

1. Use MPI_Ixxx
2. MPI_Wait for the request
3. MPI_Test

In [ ]:
%%writefile main_2.cpp
#include <mpi.h>
#include <iostream>
#include <unistd.h>

#define MESSAGE_SIZE 10000
const bool blocking0 = true;
const bool blocking1 = false;

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int myid, numprocs;
    MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
    MPI_Comm_rank(MPI_COMM_WORLD, &myid);

    if (numprocs != 2) {
        if (myid == 0) {
            std::cout << "Error: This program requires exactly 2 processors." << std::endl;
        }
        MPI_Finalize();
        return 1;
    }

    int send_data[MESSAGE_SIZE];
    int recv_data[MESSAGE_SIZE];
    for (int i = 0; i < MESSAGE_SIZE; i++) {
        send_data[i] = i + myid * MESSAGE_SIZE;
        recv_data[i] = -1;
    }
    MPI_Request send_request, recv_request;
    MPI_Status send_status, recv_status;

    if (myid == 0)
    {
        if (blocking0)
        {
          std::cout << "Processor 0 is using blocking send and receive." << std::endl;
          MPI_Send(&send_data, MESSAGE_SIZE, MPI_INT, 1, 200, MPI_COMM_WORLD);
          MPI_Recv(&recv_data, MESSAGE_SIZE, MPI_INT, 1, 100, MPI_COMM_WORLD, &recv_status);
        }
        else
        {
          std::cout << "Processor 0 is using non-blocking send and receive." << std::endl;
          MPI_Isend(&send_data, MESSAGE_SIZE, MPI_INT, 1, 200, MPI_COMM_WORLD, &send_request);
          MPI_Irecv(&recv_data, MESSAGE_SIZE, MPI_INT, 1, 100, MPI_COMM_WORLD, &recv_request);

          MPI_Wait(&send_request, &send_status);
          MPI_Wait(&recv_request, &recv_status);
        }

        std::cout << "Processor 0 sent " << send_data[0] << " and received " << recv_data[0] << std::endl;
    } else if (myid == 1)
    {
        if(blocking1)
        {
          std::cout << "Processor 1 is using blocking send and receive." << std::endl;
          MPI_Send(&send_data, MESSAGE_SIZE, MPI_INT, 0, 100, MPI_COMM_WORLD);
          MPI_Recv(&recv_data, MESSAGE_SIZE, MPI_INT, 0, 200, MPI_COMM_WORLD, &recv_status);
        }
        else
        {
          std::cout << "Processor 1 is using non-blocking send and receive." << std::endl;
          MPI_Isend(&send_data, MESSAGE_SIZE, MPI_INT, 0, 100, MPI_COMM_WORLD, &send_request);
          MPI_Irecv(&recv_data, MESSAGE_SIZE, MPI_INT, 0, 200, MPI_COMM_WORLD, &recv_request);

          MPI_Wait(&send_request, &send_status);
          MPI_Wait(&recv_request, &recv_status);
        }

        std::cout << "Processor 1 sent " << send_data[0] << " and received " << recv_data[0] << std::endl;
    }

    MPI_Finalize();
    return 0;
}

In [ ]:
!mpicxx main_2.cpp -o main_2.ex

In [ ]:
!mpirun --oversubscribe --allow-run-as-root -np 2 ./main_2.ex